## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-16-20-51-24 +0000,nyt,Text Messages From Suspect in the Kirk Shootin...,https://www.nytimes.com/2025/09/16/us/politics...
1,2025-09-16-20-47-53 +0000,nypost,Zohran Mamdani refuses to back Hochul for re-e...,https://nypost.com/2025/09/16/us-news/zohran-m...
2,2025-09-16-20-47-33 +0000,nyt,"Democrats Oppose Stopgap Bill, Raising Odds of...",https://www.nytimes.com/2025/09/16/us/politics...
3,2025-09-16-20-47-09 +0000,nyt,Here’s what to know.,https://www.nytimes.com/live/2025/09/16/us/kir...
4,2025-09-16-20-45-29 +0000,nyt,Federal Courts Want More Money From Congress f...,https://www.nytimes.com/2025/09/16/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,54
3,kirk,36
39,charlie,29
127,new,23
105,gaza,23


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
159,2025-09-16-15-56-29 +0000,nypost,Eric Trump says prez saw Charlie Kirk as ‘seco...,https://nypost.com/2025/09/16/us-news/eric-tru...,135
182,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...,126
328,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...,124
74,2025-09-16-18-31-12 +0000,bbc,Trump says he will sue New York Times for $15bn,https://www.bbc.com/news/articles/cevz419pk8vo...,116
329,2025-09-16-00-39-31 +0000,nypost,Senate confirms Trump Fed pick Stephen Miran a...,https://nypost.com/2025/09/15/us-news/senate-c...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
159,135,2025-09-16-15-56-29 +0000,nypost,Eric Trump says prez saw Charlie Kirk as ‘seco...,https://nypost.com/2025/09/16/us-news/eric-tru...
103,98,2025-09-16-17-37-00 +0000,wsj,Israel Launches New Ground Offensive Seeking t...,https://www.wsj.com/world/middle-east/israel-l...
328,70,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...
182,61,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...
1,57,2025-09-16-20-47-53 +0000,nypost,Zohran Mamdani refuses to back Hochul for re-e...,https://nypost.com/2025/09/16/us-news/zohran-m...
98,54,2025-09-16-17-49-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...
184,52,2025-09-16-14-43-00 +0000,wsj,Judge Throws Out Terror-Related Murder Charges...,https://www.wsj.com/us-news/law/judge-throws-o...
29,47,2025-09-16-19-59-00 +0000,wsj,House Republicans unveiled a spending bill tha...,https://www.wsj.com/politics/policy/government...
80,47,2025-09-16-18-22-36 +0000,nypost,Tyler Robinson grew ‘more pro-gay and trans-ri...,https://nypost.com/2025/09/16/us-news/tyler-ro...
199,41,2025-09-16-14-02-24 +0000,nypost,Russia launches large-scale attack on Ukrainia...,https://nypost.com/2025/09/16/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
